<a href="https://colab.research.google.com/github/Dhaneshkp/NLP/blob/main/llm_classification_v1_open-minilm-l6-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:


#!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer

#model = SentenceTransformer("all-mpnet-base-v2")


In [ ]:

#model.save("/kaggle/working/all-mpnet-base-v2")


In [ ]:
from transformers import AutoModel, AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/qwen-llm")
#model = AutoModel.from_pretrained("/kaggle/input/qwen-llm")



In [ ]:
df_train = pd.read_csv("/content/train.csv")
df_train

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0
...,...,...,...,...,...,...,...,...,...
57472,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0
57473,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0
57474,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0
57475,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0


In [ ]:
df_test = pd.read_csv("/content/test.csv")
df_test

,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p..."


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57477 entries, 0 to 57476
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              57477 non-null  int64 
 1   model_a         57477 non-null  object
 2   model_b         57477 non-null  object
 3   prompt          57477 non-null  object
 4   response_a      57477 non-null  object
 5   response_b      57477 non-null  object
 6   winner_model_a  57477 non-null  int64 
 7   winner_model_b  57477 non-null  int64 
 8   winner_tie      57477 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 3.9+ MB


In [ ]:
df_train.isnull().sum()

,0
id,0
model_a,0
model_b,0
prompt,0
response_a,0
response_b,0
winner_model_a,0
winner_model_b,0
winner_tie,0


In [ ]:
X = df_train.drop(['model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie'], axis = 1)
X

,id,prompt,response_a,response_b
0,30192,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o..."
1,53567,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat..."
2,65089,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ..."
3,96401,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c..."
4,198779,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru..."
...,...,...,...,...
57472,4294656694,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig..."
57473,4294692063,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba..."
57474,4294710549,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us..."
57475,4294899228,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or..."


In [ ]:
y = df_train[['winner_model_a', 'winner_model_b', 'winner_tie']].values

y = np.argmax(y, axis=1)
y

array([0, 1, 2, ..., 0, 1, 0])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 42)

In [ ]:
X_train

,id,prompt,response_a,response_b
30205,2245506736,"[""can you give me some escape room themes with...","[""Certainly! Here are several escape room them...","[""Here are some fun and immersive escape room ..."
8763,656268525,"[""Why did digital electronics switch from 5 vo...","[""There are a few key reasons why digital elec...","[""The shift from 5 volt logic to 3 volt logic ..."
30501,2266396135,"[""as an acclaimed and visionary author punishe...","[""As an acclaimed and visionary author, Elieze...","[""Eliezer Yudkowsky is often referred to as th..."
33619,2497468695,"[""\n\nWrite a micro editor plugin that shows t...","[""Here is Python code for a basic micro editor...","[""Micro is a modern and intuitive terminal-bas..."
22573,1679992057,"[""1. 19 year old Emese who is thin (165cm 48kg...","[""Certainly, let's address each of your points...","[""1. From the description you provided, it see..."
...,...,...,...,...
54343,4061186081,"[""mega-sena brazilian lottery statistics""]","[""I do not have access to real-time mega-sena ...","[""Mega-Sena is a Brazilian lottery game that o..."
38158,2840606458,"[""Draw a dog using ASCII art""]","[""\/_\/\\ \n( o.o )\n> ^""]","[""Here is a dog drawn with ASCII art:\n\n ..."
860,62750940,"[""How to wear a bikini?""]","[""Wearing a bikini can be a fun and comfortabl...","[""Here are the basic steps to wear a bikini:\n..."
15795,1180878567,"[""Can you tell me what was going on with all t...","[""The direwolves, which are also known as dire...","[""Sure! On the night of the Red Wedding, all o..."


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
X_train["response_a_clean"] = X_train["response_a"].apply(clean_text)
X_train["response_b_clean"] = X_train["response_b"].apply(clean_text)

X_train["prompt_clean"] = X_train["prompt"].apply(clean_text)


In [ ]:
X_test

,id,prompt,response_a,response_b
37379,2785062085,"[""what does hello world mean""]","[""\""Hello, World!\"" is a phrase used in comput...","[""\""Hello, World!\"" is a common phrase used to..."
644,48259531,"[""I ran a marathon in 3:12:00 weighting 84kg. ...","[""It's difficult to provide an exact answer to...","[""To accurately estimate how much faster you w..."
48496,3622731894,"[""Below is an instruction that describes a tas...","[""\u8fd9\u6b3e\u7eaf\u8272\u4e94\u5206\u88e4\u...","[""\u6839\u636e\u63d0\u4f9b\u7684\u4ea7\u54c1\u..."
12603,933663766,"[""How do I run static analysis with gcc in QT ...","[""To run static analysis with gcc in QT Creato...","[""To run static analysis with gcc in QT Creato..."
16697,1246768370,"[""What did the music producer \u00d8fdream die...","[""I am not able to provide information on the ...","[""I'm really sorry, but I don't have the abili..."
...,...,...,...,...
3151,240261313,"[""How to save a web page into a jpg file?""]","[""Here are a few ways to save a web page as a ...","[""Here are a few ways to save a web page as a ..."
30983,2300512815,"[""provide a short story about a pirate that le...","[""Here is a short story about a pirate who lea...","[""Once upon a time, in the golden age of pirac..."
31183,2314277410,"[""Name 3 features of Alain Badiou's ontology t...","[""Alain Badiou and Plato have distinct philoso...","[""Here are 3 key differences between Alain Bad..."
50634,3781372084,"[""Please give me a list of concise bullet poin...","[""- Musician who plays guitar and enjoys jammi...","[""* Music: plays guitar, drums, and bass\n* Sp..."


In [ ]:
y_train

array([2, 1, 1, ..., 1, 1, 0])

In [ ]:
catagorical_feature = [col for col in X.columns if X[col].dtype == 'object']
catagorical_feature

['prompt', 'response_a', 'response_b']

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2


from sentence_transformers import SentenceTransformer
import torch

from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2


from sentence_transformers import SentenceTransformer
import torch

class HFEmbedder(BaseEstimator, TransformerMixin):
    def __init__(self, model_name="all-mpnet-base-v2"):
        self.model_name = model_name  
        self.model = SentenceTransformer(model_name)
        if torch.cuda.is_available():
            self.model = self.model.to("cuda")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return self.model.encode(X.tolist(), show_progress_bar=False, device="cuda" if torch.cuda.is_available() else "cpu")


# Example usage in ColumnTransformer
preprocessing = ColumnTransformer([
    ("prompt_embed", HFEmbedder(), "prompt"),
    ("resp_a_embed", HFEmbedder(), "response_a"),
    ("resp_b_embed", HFEmbedder(), "response_b"),
    ("num", "passthrough", ["id"])
])


from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2


from sentence_transformers import SentenceTransformer
import torch
import torch

class HFEmbedder(BaseEstimator, TransformerMixin):
    def __init__(self, model_path="/kaggle/input/open-minilm-l6-v2",use_auth_token=False , local_files_only=True,batch_size=8, use_cuda=True):
        self.model_path = model_path
        self.batch_size = batch_size
        self.use_cuda = use_cuda and torch.cuda.is_available()
        self.model = SentenceTransformer(self.model_path)
        if self.use_cuda:
            self.model = self.model.to("cuda")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        texts = X.tolist()
        all_embeddings = []

        for i in range(0, len(texts), self.batch_size):
            batch = texts[i:i + self.batch_size]
            embeddings = self.model.encode(
                batch,
                show_progress_bar=False,
                device="cuda" if self.use_cuda else "cpu"
            )
            all_embeddings.append(embeddings)

        return np.vstack(all_embeddings)



from transformers import AutoTokenizer, AutoModel
from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np
import keras_nlp
import keras

from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import keras_nlp

class HFEmbedder(BaseEstimator, TransformerMixin):
    def __init__(self, weights_path="/kaggle/input/dbertav3-small/model.weights.h5", batch_size=8):
        self.batch_size = batch_size
        self.backbone = keras_nlp.models.DebertaV3Backbone.from_preset("deberta_v3_extra_small_en")
        self.backbone.load_weights(weights_path)
        self.preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset("deberta_v3_extra_small_en")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        texts = X.tolist()
        all_embeddings = []

        for i in range(0, len(texts), self.batch_size):
            batch = texts[i:i + self.batch_size]
            tokenized = self.preprocessor(batch)
            outputs = self.backbone(tokenized)
            embeddings = np.mean(outputs["sequence_output"].numpy(), axis=1)
            all_embeddings.append(embeddings)

        return np.vstack(all_embeddings)



import shutil

shutil.copy("/kaggle/input/dbertav3-small/vocabulary.spm", "/kaggle/working/spm.model")


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

class HFEmbedder(BaseEstimator, TransformerMixin):
    def __init__(self, model_path="sentence-transformers/multi-qa-mpnet-base-cos-v1", batch_size=8, use_cuda=True):
        self.model_path = model_path
        self.batch_size = batch_size
        self.use_cuda = use_cuda and torch.cuda.is_available()
        self.model = SentenceTransformer(self.model_path)
        if self.use_cuda:
            self.model = self.model.to("cuda")

    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # Ensure input is a list of strings
        if isinstance(X, (np.ndarray, list)):
            texts = list(X)
        elif hasattr(X, "tolist"):
            texts = X.tolist()
        else:
            raise ValueError("Unsupported input type for HFEmbedder")

        all_embeddings = self.model.encode(
            texts,
            show_progress_bar=False,
            convert_to_numpy=True,
            batch_size=self.batch_size,
            device="cuda" if self.use_cuda else "cpu"
        )

        return all_embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
class PromptResponseSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, model_path="sentence-transformers/multi-qa-mpnet-base-cos-v1", batch_size=8, use_cuda=True):
        self.model_path = model_path
        self.batch_size = batch_size
        self.use_cuda = use_cuda and torch.cuda.is_available()

        self.model = SentenceTransformer(self.model_path)
        if self.use_cuda:
            self.model = self.model.to("cuda")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        prompts = X["prompt_clean"].tolist()
        responses_a = X["response_a_clean"].tolist()
        responses_b = X["response_b_clean"].tolist()

        def embed(texts):
             embeddings = self.model.encode(
                texts,
                show_progress_bar=False,
                convert_to_numpy=True,
                batch_size=self.batch_size,
                device="cuda" if self.use_cuda else "cpu"
            )
             return embeddings


        prompt_embeds = embed(prompts)
        a_embeds = embed(responses_a)
        b_embeds = embed(responses_b)

        sim_a = cosine_similarity(prompt_embeds, a_embeds).diagonal()
        sim_b = cosine_similarity(prompt_embeds, b_embeds).diagonal()

        return np.vstack([sim_a, sim_b]).T


embedder = HFEmbedder(model_path="Qwen/Qwen3-Embedding-0.6B")
X_train_embed = X_train.copy()
X_train_embed["prompt_embed"] = embedder.transform(X_train["prompt"])
X_train_embed["resp_a_embed"] = embedder.transform(X_train["response_a"])
X_train_embed["resp_b_embed"] = embedder.transform(X_train["response_b"])

# Drop original text columns if not needed
X_train_embed = X_train_embed.drop(columns=["prompt", "response_a", "response_b"])


from transformers import AutoTokenizer, AutoModel
from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np

class HFEmbedder(BaseEstimator, TransformerMixin):
    def __init__(self, model_path="/kaggle/input/dbertav3-small", use_fast=False,batch_size=8, use_cuda=True):
        self.model_path = model_path
        self.batch_size = batch_size
        self.use_cuda = use_cuda and torch.cuda.is_available()
        #self.tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/dbertav3-small", use_fast=False)

        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        self.model = AutoModel.from_pretrained(self.model_path)
        if self.use_cuda:
            self.model = self.model.to("cuda")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        texts = X.tolist()
        all_embeddings = []

        for i in range(0, len(texts), self.batch_size):
            batch = texts[i:i + self.batch_size]
            inputs = self.tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
            if self.use_cuda:
                inputs = {k: v.to("cuda") for k, v in inputs.items()}
            with torch.no_grad():
                outputs = self.model(**inputs)
            # Mean pooling over token embeddings
            embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
            all_embeddings.append(embeddings)

        return np.vstack(all_embeddings)


from transformers import AutoTokenizer, AutoModel
from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

class PromptResponseSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, model_path="/kaggle/input/open-minilm-l6-v2", batch_size=8, use_cuda=True):
        self.model_path = model_path
        self.batch_size = batch_size
        self.use_cuda = use_cuda and torch.cuda.is_available()

        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        self.model = AutoModel.from_pretrained(self.model_path)
        if self.use_cuda:
            self.model = self.model.to("cuda")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        prompts = X["prompt"].tolist()
        responses_a = X["response_a"].tolist()
        responses_b = X["response_b"].tolist()


        def embed(texts):
            embeddings = []
            for i in range(0, len(texts), self.batch_size):
                batch = texts[i:i + self.batch_size]
                inputs = self.tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
                if self.use_cuda:
                    inputs = {k: v.to("cuda") for k, v in inputs.items()}
                with torch.no_grad():
                    outputs = self.model(**inputs)
                batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
                embeddings.append(batch_embeddings)
            return np.vstack(embeddings)

        prompt_embeds = embed(prompts)
        a_embeds = embed(responses_a)
        b_embeds = embed(responses_b)

        sim_a = cosine_similarity(prompt_embeds, a_embeds).diagonal()
        sim_b = cosine_similarity(prompt_embeds, b_embeds).diagonal()

        return np.vstack([sim_a, sim_b]).T


print(os.listdir("/kaggle/input/open-minilm-l6-v2"))



from sklearn.compose import ColumnTransformer
preprocessing = ColumnTransformer([
    ("prompt_embed", HFEmbedder("/kaggle/input/dbertav3-small"), "prompt"),
    ("resp_a_embed", HFEmbedder("/kaggle/input/dbertav3-small"), "response_a"),
    ("resp_b_embed", HFEmbedder("/kaggle/input/dbertav3-small"), "response_b"),
    ("num", "passthrough", ["id"])
])



from sklearn.compose import ColumnTransformer

# Define the embedder with the desired model path
embedder = HFEmbedder(model_path="sentence-transformers/all-MiniLM-L6-v2")
preprocessing = ColumnTransformer([
    ("sim_diff_squared", PromptResponseSimilarity("/kaggle/input/open-minilm-l6-v2"), ["prompt_clean", "response_a_clean", "response_b_clean"]),
    ("prompt_embed", embedder, "prompt_clean"),
    ("resp_a_embed", embedder, "response_a_clean"),
    ("resp_b_embed", embedder, "response_b_clean"),
])



preprocessing = ColumnTransformer([
    ("similarities", PromptResponseSimilarity("/kaggle/input/open-minilm-l6-v2"), ["prompt", "response_a", "response_b"]),
    ("prompt_embed", HFEmbedder("/kaggle/input/open-minilm-l6-v2"), "prompt"),
    ("resp_a_embed", HFEmbedder("/kaggle/input/open-minilm-l6-v2"), "response_a"),
    ("resp_b_embed", HFEmbedder("/kaggle/input/open-minilm-l6-v2"), "response_b"),
    ("num", "passthrough", ["id"])
])


preprocessing

In [ ]:
#feature_selection = SelectKBest(score_func = chi2, k=6)
from sklearn.feature_selection import SelectKBest, f_classif

feature_selection = SelectKBest(score_func=f_classif, k=6)


In [ ]:
model = XGBClassifier(
    objective="multi:softprob",
    num_class=3,
    eval_metric="mlogloss",
    n_estimators=300,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sentence_transformers import SentenceTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Use HFEmbedder and CommonWordsTransformer
embedder = HFEmbedder(model_path="sentence-transformers/multi-qa-mpnet-base-cos-v1")
common_words_transformer = CommonWordsTransformer()

# Combine transformers using ColumnTransformer
# Make sure to include the columns needed by each transformer
feature_union = ColumnTransformer(
    transformers=[
        ("prompt_embed", embedder, "prompt_clean"),
        ("resp_a_embed", embedder, "response_a_clean"),
        ("resp_b_embed", embedder, "response_b_clean"),
        ("common_words", common_words_transformer, ["prompt_clean", "response_a_clean", "response_b_clean"]),
    ]
)

# Define the model (assuming XGBClassifier is defined in a previous cell)
# from your_model import model

# Create the final pipeline
my_pipeline = Pipeline([
    ("features", feature_union),
    # Adjust indexing for common_words_diff based on the new feature_union output
    ("common_words_diff", FunctionTransformer(lambda X: (X[:, -2] - X[:, -1]).reshape(-1, 1))),
    ("model", model)
])

display(my_pipeline)

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('prompt_embed', HFEmbedder(),
                                                  'prompt_clean'),
                                                 ('resp_a_embed', HFEmbedder(),
                                                  'response_a_clean'),
                                                 ('resp_b_embed', HFEmbedder(),
                                                  'response_b_clean'),
                                                 ('common_words',
                                                  CommonWordsTransformer(),
                                                  ['prompt_clean',
                                                   'response_a_clean',
                                                   'response_b_clean'])])),
                ('common_words_diff',
                 FunctionTransformer(func=<...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None, num_class=3, ...))])

In [ ]:
scores = cross_val_score(my_pipeline, X_train, y_train, cv=2, scoring="accuracy")
print("Accuracy:", scores.mean())

Accuracy: 0.3741980034966119


In [ ]:
my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('prompt_embed', HFEmbedder(),
                                                  'prompt_clean'),
                                                 ('resp_a_embed', HFEmbedder(),
                                                  'response_a_clean'),
                                                 ('resp_b_embed', HFEmbedder(),
                                                  'response_b_clean'),
                                                 ('common_words',
                                                  CommonWordsTransformer(),
                                                  ['prompt_clean',
                                                   'response_a_clean',
                                                   'response_b_clean'])])),
                ('common_words_diff',
                 FunctionTransformer(func=<...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None, num_class=3, ...))])

In [ ]:
df_test["response_a_clean"] = df_test["response_a"].apply(clean_text)
df_test["response_b_clean"] = df_test["response_b"].apply(clean_text)

df_test["prompt_clean"] = df_test["prompt"].apply(clean_text)

In [ ]:
df_test

,id,prompt,response_a,response_b,response_a_clean,response_b_clean,prompt_clean
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran...",two orange today,still three orange eating orange yesterday aff...,three orange today ate orange yesterday many o...
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ...",thank sharing detail situation mediator unders...,mr reddy m blue valid point argument one hand ...,mediator heated political debate two opposing ...
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p...",want initialize classification head transfer l...,initialize classification head performing tran...,initialize classification head transfer learni...


In [ ]:
probs = my_pipeline.predict_proba(df_test)

submission = pd.DataFrame({
    "id": df_test["id"],
    "winner_model_a": probs[:,0],
    "winner_model_b": probs[:,1],
    "winner_tie": probs[:,2],
})
print(submission)
submission.to_csv("submission.csv", index=False)

        id  winner_model_a  winner_model_b  winner_tie
0   136060        0.313254        0.389765    0.296981
1   211333        0.328896        0.363026    0.308078
2  1233961        0.373386        0.319660    0.306954


In [ ]:
df_test

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text_for_common_words(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.split()
    return [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]


class CommonWordsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        common_meaningful_words_a = []
        common_meaningful_words_b = []

        for index, row in X.iterrows():
            prompt_tokens = clean_text_for_common_words(row['prompt_clean'])
            response_a_tokens = clean_text_for_common_words(row['response_a_clean'])
            response_b_tokens = clean_text_for_common_words(row['response_b_clean'])

            common_meaningful_a = len(set(prompt_tokens) & set(response_a_tokens))
            common_meaningful_b = len(set(prompt_tokens) & set(response_b_tokens))

            common_meaningful_words_a.append(common_meaningful_a)
            common_meaningful_words_b.append(common_meaningful_b)

        return np.array([common_meaningful_words_a, common_meaningful_words_b]).T

In [ ]:
X_train["response_a_clean"] = X_train["response_a"].apply(clean_text)
X_train["response_b_clean"] = X_train["response_b"].apply(clean_text)
X_train["prompt_clean"] = X_train["prompt"].apply(clean_text)

X_test["response_a_clean"] = X_test["response_a"].apply(clean_text)
X_test["response_b_clean"] = X_test["response_b"].apply(clean_text)
X_test["prompt_clean"] = X_test["prompt"].apply(clean_text)


common_words_transformer = CommonWordsTransformer()

X_train_common_words = common_words_transformer.transform(X_train)
X_test_common_words = common_words_transformer.transform(X_test)

X_train['common_meaningful_a'] = X_train_common_words[:, 0]
X_train['common_meaningful_b'] = X_train_common_words[:, 1]

X_test['common_meaningful_a'] = X_test_common_words[:, 0]
X_test['common_meaningful_b'] = X_test_common_words[:, 1]

display(X_train.head())
display(X_test.head())

,id,prompt,response_a,response_b,response_a_clean,response_b_clean,prompt_clean,common_meaningful_a,common_meaningful_b
30205,2245506736,"[""can you give me some escape room themes with...","[""Certainly! Here are several escape room them...","[""Here are some fun and immersive escape room ...",certainly several escape room theme create imm...,fun immersive escape room theme ideasnn ancien...,give escape room theme immersive atmosphere,5,4
8763,656268525,"[""Why did digital electronics switch from 5 vo...","[""There are a few key reasons why digital elec...","[""The shift from 5 volt logic to 3 volt logic ...",key reason digital electronics switched 5 volt...,shift 5 volt logic 3 volt logic digital electr...,digital electronics switch 5 volt logic 3 volt...,10,11
30501,2266396135,"[""as an acclaimed and visionary author punishe...","[""As an acclaimed and visionary author, Elieze...","[""Eliezer Yudkowsky is often referred to as th...",acclaimed visionary author eliezer yudkowsky k...,eliezer yudkowsky often referred ayn rand time...,acclaimed visionary author punished top magazi...,8,5
33619,2497468695,"[""\n\nWrite a micro editor plugin that shows t...","[""Here is Python code for a basic micro editor...","[""Micro is a modern and intuitive terminal-bas...",python code basic micro editor plugin display ...,micro modern intuitive terminalbased text edit...,nnwrite micro editor plugin show text hello wo...,6,6
22573,1679992057,"[""1. 19 year old Emese who is thin (165cm 48kg...","[""Certainly, let's address each of your points...","[""1. From the description you provided, it see...",certainly let address point structured manner ...,1 description provided seems like emese dealin...,1 19 year old emese thin 165cm 48kg pain kneec...,40,23


,id,prompt,response_a,response_b,response_a_clean,response_b_clean,prompt_clean,common_meaningful_a,common_meaningful_b
37379,2785062085,"[""what does hello world mean""]","[""\""Hello, World!\"" is a phrase used in comput...","[""\""Hello, World!\"" is a common phrase used to...",hello world phrase used computer programming i...,hello world common phrase used display message...,hello world mean,2,2
644,48259531,"[""I ran a marathon in 3:12:00 weighting 84kg. ...","[""It's difficult to provide an exact answer to...","[""To accurately estimate how much faster you w...",difficult provide exact answer question many f...,accurately estimate much faster would run mara...,ran marathon 31200 weighting 84kg fast would r...,4,4
48496,3622731894,"[""Below is an instruction that describes a tas...","[""\u8fd9\u6b3e\u7eaf\u8272\u4e94\u5206\u88e4\u...","[""\u6839\u636e\u63d0\u4f9b\u7684\u4ea7\u54c1\u...",u8fd9u6b3eu7eafu8272u4e94u5206u88e4u8bbeu8ba1u...,u6839u636eu63d0u4f9bu7684u4ea7u54c1u98ceu683cu...,instruction describes task paired input provid...,0,0
12603,933663766,"[""How do I run static analysis with gcc in QT ...","[""To run static analysis with gcc in QT Creato...","[""To run static analysis with gcc in QT Creato...",run static analysis gcc qt creator follow step...,run static analysis gcc qt creator follow step...,run static analysis gcc qt creator,6,6
16697,1246768370,"[""What did the music producer \u00d8fdream die...","[""I am not able to provide information on the ...","[""I'm really sorry, but I don't have the abili...",able provide information cause u00d8fdreams de...,im really sorry dont ability access realtime p...,music producer u00d8fdream die,0,3
